In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# 1. 데이터 불러오기 ---------------------------------------------------------
df = pd.read_csv("stroke.csv")

# 2. 사용할 독립변수(피처) 목록 정의 ----------------------------------------
features = [
    "age", "avg_glucose_level", "bmi",
    "gender", "hypertension", "heart_disease",
    "ever_married", "work_type", "Residence_type",
    "smoking_status"
]

# 3. X(독립변수), y(종속변수) 만들기 ----------------------------------------
X = df[features].copy()
y = df["stroke"]

# 4. 범주형 변수 → 더미 변수(one-hot), 첫 카테고리는 기준으로 드랍 ---------
cat_cols = ["gender", "ever_married", "work_type", "Residence_type", "smoking_status"]
X = pd.get_dummies(X, columns=cat_cols, drop_first=True)

# 5. 상수항 추가 ------------------------------------------------------------
X = sm.add_constant(X)

# =====================================================================
# ✅ 1. VIF 계산 (다중공선성 확인)
# =====================================================================
vif_df = pd.DataFrame()
vif_df["variable"] = X.columns
vif_df["VIF"] = [variance_inflation_factor(int(X.values), i) for i in range(X.shape[1])]

print("=== VIF 결과 ===")
print(vif_df)

# =====================================================================
# ✅ 2. Multivariable Logistic Regression (모든 변수 포함)
# =====================================================================
logit_model = sm.Logit(y, X)
result = logit_model.fit()   # disp=0 주면 로그 안 나오게 할 수 있음

# 기본 회귀 결과 요약
print(result.summary())

# =====================================================================
# ✅ 3. Adjusted OR, 95% CI, p-value 정리
# =====================================================================
params = result.params               # 회귀계수 β
conf_int = result.conf_int()         # 95% CI (β 기준)
p_values = result.pvalues            # p-value

or_df = pd.DataFrame({
    "variable": params.index,
    "OR": np.exp(params),            # Odds Ratio = exp(β)
    "CI_lower": np.exp(conf_int[0]), # 95% CI 하한 (exp)
    "CI_upper": np.exp(conf_int[1]), # 95% CI 상한 (exp)
    "p_value": p_values
})

print("=== Adjusted OR (다변량 로지스틱) ===")
print(or_df)


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()